In [28]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [29]:
link = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(link,'lxml')

In [30]:
table_can_zipinfo = soup.find('table')
colvals = table_can_zipinfo.find_all('td')

elem_cnt = len(colvals)

postcode = []
borough = []
neighborhood = []

for i in range(0, elem_cnt, 3):
    postcode.append(colvals[i].text.strip())
    borough.append(colvals[i+1].text.strip())
    neighborhood.append(colvals[i+2].text.strip())

In [31]:
tordf = pd.DataFrame(data=[postcode, borough, neighborhood]).transpose()
tordf.columns = ['Postcode', 'Borough', 'Neighborhood']

In [32]:
tordf.drop(tordf[tordf['Borough'] == 'Not assigned'].index, inplace=True)
tordf.loc[tordf.Neighborhood == 'Not assigned', "Neighborhood"] = tordf.Borough

In [14]:
tordf_group.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [33]:
tordf_group = tordf.groupby(['Postcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
tordf_group.columns = ['Postcode', 'Borough', 'Neighborhood']

In [34]:
torcor = pd.read_csv('https://cocl.us/Geospatial_data')
torcor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [43]:
tordf_group['Latitude'] = ""
tordf_group['Longitude'] = ""
tordf_group.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",,
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",,
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",,
3,M1G,Scarborough,Woburn,,
4,M1H,Scarborough,Cedarbrae,,


In [44]:
for idx in tordf_group.index:
    torcoridx = torcor['Postal Code'] == tordf_group.loc[idx, 'Postcode']
    tordf_group.at[idx, 'Latitude'] = torcor.loc[torcoridx, 'Latitude'].values
    tordf_group.at[idx, 'Longitude'] = torcor.loc[torcoridx, 'Longitude'].values

In [45]:
tordf_group.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",[43.806686299999996],[-79.19435340000001]
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",[43.7845351],[-79.16049709999999]
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",[43.7635726],[-79.1887115]
3,M1G,Scarborough,Woburn,[43.7709921],[-79.21691740000001]
4,M1H,Scarborough,Cedarbrae,[43.773136],[-79.23947609999999]
